In [2]:
import jutil
import numpy as np
from collections import Counter
import pyfpgrowth

In [3]:
sql = "select t.Course_code as course_code from train_plan_course t,train_plan_credit c where t.Course_big_type_id=c.sid \
and t.Grade='2014' and c.type_name like '%专业%'  and t.Speciality_code='0408'"
code = [d['course_code'] for d in jutil.load_data(sql)]

In [4]:
len(set(code))

53

In [5]:
sql = "select student_id,course_name,course_code,pmark,mark from view_stu_course_mark where speciality_code='0408' and grade='2014'"
df = jutil.load_pd_df(sql)
df.head()

,course_code,course_name,mark,pmark,student_id
0,0471004,面向对象程序设计,9,88.0,20122639
1,0471109,软件工程/程序综合设计实习,10,91.0,20122639
2,6010500,线性代数B,10,94.0,20122639
3,6041943,计算方法C,8,84.0,20122639
4,7001153,形势与政策4,10,91.0,20122639


In [6]:
# 选出了与专业相关的课程
dd = df[df['course_code'].isin(code)]

## 按学生group,然后将其课程分数作为维度,学生数作为数据量

In [7]:
code = dd.groupby(['student_id'],as_index=False)['course_code'].apply(list)
mark = dd.groupby(['student_id'],as_index=False)['mark'].apply(list)

In [8]:
d_mark = mark.to_dict()
d_code = code.to_dict()

In [9]:
transac = []
for k,cd in d_code.items():
    mk = d_mark.get(k,[])
    l = len(mk) if len(mk)<20 else 20
    arr = []
#     for i in range(len(mk)):
    for i in range(l):
        if mk[i]>'1':
            arr.append(cd[i]+'_0')
        else:
            arr.append(cd[i]+'_1')
#     arr = [cd[i]+'_'+mk[i] for i in range(len(mk))]
    transac.append(arr)

In [10]:
from pymining import itemmining,assocrules

In [11]:
relim_input = itemmining.get_relim_input(transac)

In [12]:
item_sets = itemmining.relim(relim_input, min_support=5)

In [13]:
ules = assocrules.mine_assoc_rules(item_sets, min_support=5, min_confidence=0.7)

In [14]:
ules

[(frozenset({'0471005_0',
             '0471024_0',
             '0471075_0',
             '3200557_0',
             '6010500_0',
             '6041943_0'}),
  frozenset({'0471046_0'}),
  5,
  1.0),
 (frozenset({'0471005_0', '0471024_0', '0471075_0', '3200557_0', '6041943_0'}),
  frozenset({'0471046_0', '6010500_0'}),
  5,
  0.8333333333333334),
 (frozenset({'0471005_0', '0471024_0', '3200557_0', '6041943_0'}),
  frozenset({'0471046_0', '0471075_0', '6010500_0'}),
  5,
  0.7142857142857143),
 (frozenset({'0471005_0', '0471024_0', '0471075_0', '3200557_0'}),
  frozenset({'0471046_0', '6010500_0', '6041943_0'}),
  5,
  0.7142857142857143),
 (frozenset({'0471005_0', '0471075_0', '3200557_0', '6041943_0'}),
  frozenset({'0471024_0', '0471046_0', '6010500_0'}),
  5,
  0.7142857142857143),
 (frozenset({'0471024_0', '0471075_0', '3200557_0', '6041943_0'}),
  frozenset({'0471005_0', '0471046_0', '6010500_0'}),
  5,
  0.8333333333333334),
 (frozenset({'0471005_0', '0471024_0', '3200557_0', '601

In [19]:
transac

[['0471072_0',
  '0473082_0',
  '0474020_0',
  '3244238_0',
  '6011320_0',
  '6023200_0',
  '0471004_0',
  '6010500_0',
  '6041943_0',
  '0471054_0',
  '0473031_0',
  '3045600_0',
  '3243020_0',
  '6111110_0',
  '9990501_0',
  '0474003_0',
  '0474012_0',
  '0474014_0',
  '0474019_0',
  '3243781_0',
  '0471005_0',
  '0471046_0',
  '0471075_0',
  '0471076_0',
  '3200557_0',
  '3100312_0',
  '3200510_0',
  '3232400_0',
  '3243120_0',
  '3243510_0',
  '6033100_0',
  '9990620_0',
  '3143300_0',
  '3242400_0',
  '6011310_0',
  '6100131_0',
  '6111120_0',
  '0471024_0',
  '0471071_0',
  '0473041_0',
  '0474011_0',
  '3232100_0',
  '3243700_0',
  '3244152_0'],
 ['0471076_1',
  '0473041_0',
  '3200557_0',
  '3243120_0',
  '0474020_0',
  '3045600_0',
  '3100312_0',
  '3232100_1',
  '3243020_1',
  '3244238_0',
  '0471004_1',
  '0471046_0',
  '0474014_0',
  '3243510_0',
  '6011320_1',
  '6011320_1',
  '0471005_1',
  '0471071_1',
  '0474011_0',
  '3243700_0',
  '6010500_1',
  '0471071_1',
  '047401

In [16]:
p = pyfpgrowth.find_frequent_patterns(transactions=transac,support_threshold=5)

In [17]:
pyfpgrowth.generate_association_rules(p,0.7)

{('6111120_1',): (('0471076_0',), 1.0),
 ('6011310_1',): (('6010500_1',), 1.6666666666666667),
 ('0471004_1', '0474019_0'): (('3243781_0',), 0.8333333333333334),
 ('0471004_1', '3243700_0'): (('0474019_0', '3243781_0'), 1.0),
 ('0471004_1', '3243781_0'): (('0474019_0',), 0.8333333333333334),
 ('0471004_1', '0474019_0', '3243700_0'): (('3243781_0',), 1.0),
 ('0471004_1', '0474019_0', '3243781_0'): (('3243700_0',), 1.0),
 ('0471004_1', '3243700_0', '3243781_0'): (('0474019_0',), 1.0),
 ('0471004_1', '6033100_0'): (('3243781_0',), 1.0),
 ('0474012_0', '6100131_1'): (('0471075_1',), 0.8333333333333334),
 ('6033100_1', '6100131_1'): (('0474012_0',), 1.0),
 ('0471075_1', '6100131_1'): (('0471072_0',), 1.0),
 ('0471072_0', '6100131_1'): (('0471075_1',), 1.0),
 ('0471071_1', '0471072_0'): (('0471046_0', '3243510_0'), 0.7142857142857143),
 ('0471071_1', '3232100_1'): (('0471072_0',), 1.0),
 ('0471072_0', '3232100_1'): (('0473082_0',), 0.7142857142857143),
 ('0471071_1', '0471075_1'): (('0471072

In [6]:
transactions = [[1, 2, 5],
                [2, 4],
                [2, 3],
                [1, 2, 4],
                [1, 3],
                [2, 3],
                [1, 3],
                [1, 2, 3, 5],
                [1, 2, 3]]

p = pyfpgrowth.find_frequent_patterns(transactions=transactions,support_threshold=2)

In [8]:
pyfpgrowth.generate_association_rules(p,0.7)

{(1, 5): ((2,), 1.0),
 (2, 5): ((1,), 1.0),
 (4,): ((2,), 1.0),
 (5,): ((1, 2), 1.0)}